In [ ]:
import tensorflow as tf
from matplotlib import pyplot
from tensorflow.keras.datasets import cifar10
from emnist import extract_training_samples
from tensorflow.keras import *
from tensorflow.keras.layers import *
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt
from pprint import pprint
import sys
import os
import json
from sklearn.utils import resample

sys.path.insert(1, os.path.join(sys.path[0], '../../..'))

from utils import *
# tf.compat.v1.enable_eager_execution()
%matplotlib inline
%config Completer.use_jedi = False

tf.config.list_physical_devices('GPU')

In [ ]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
show_dataset(x_train)

In [ ]:
num_classes = 10 

mean = np.mean(x_train,axis=(0, 1, 2, 3))
std = np.std(x_train, axis=(0, 1, 2, 3))

x_train = (x_train-mean)/(std+1e-7)
x_test = (x_test-mean)/(std+1e-7)

y_train = utils.to_categorical(y_train, num_classes) 
y_test = utils.to_categorical(y_test, num_classes) 

In [ ]:
x_train.shape

In [ ]:
results = {}

# Supernet BENN

In [ ]:
import tensorflow as tf
import larq as lq

kwargs = dict(input_quantizer="ste_sign",
              kernel_quantizer="ste_sign",
              kernel_constraint="weight_clip",
              use_bias=False)
 
def get_model():    
    model = tf.keras.models.Sequential([
        lq.layers.QuantConv2D(32, 3,
                              kernel_quantizer="ste_sign",
                              kernel_constraint="weight_clip",
                              use_bias=False,
                              input_shape=(32, 32, 3)),
        tf.keras.layers.Activation("elu"),
        tf.keras.layers.BatchNormalization(),

        lq.layers.QuantConv2D(32, 3, padding="same", **kwargs),
        tf.keras.layers.Activation("elu"),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.MaxPool2D(pool_size=(2, 2)),
    #     tf.keras.layers.Dropout(0.4),

        lq.layers.QuantConv2D(64, 3, padding="same", **kwargs),
        tf.keras.layers.Activation("elu"),
        tf.keras.layers.BatchNormalization(),

        lq.layers.QuantConv2D(64, 3, padding="same", **kwargs),
        tf.keras.layers.Activation("elu"),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.MaxPool2D(pool_size=(2, 2)),
    #     tf.keras.layers.Dropout(0.4),

        lq.layers.QuantConv2D(128, 3, padding="same", **kwargs),
        tf.keras.layers.Activation("elu"),
        tf.keras.layers.BatchNormalization(),

        lq.layers.QuantConv2D(128, 3, padding="same", **kwargs),
        tf.keras.layers.Activation("elu"),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.MaxPool2D(pool_size=(2, 2)),
    #     tf.keras.layers.Dropout(0.4),


        tf.keras.layers.Flatten(),

        lq.layers.QuantDense(128, **kwargs),
        tf.keras.layers.Activation("elu"),
        tf.keras.layers.BatchNormalization(),


        lq.layers.QuantDense(10, **kwargs),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Activation("softmax")
    ])
    
    model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
    return model

In [ ]:
# history = model.fit(x_train, y_train, batch_size=256, epochs=150, validation_data=(x_test, y_test))

In [ ]:
members = []
n = len(x_train)

In [ ]:
for i in range(8):
    members.append(tf.keras.models.load_model('vanilla_cifar10_bnn_{}.h5'.format(i+1)))

In [ ]:
from tensorflow.keras.models import Model

In [ ]:
# members[7].summary()

In [ ]:
# model1= Model(inputs=members[0].input, outputs=members[0].get_layer('quant_dense_24').output)
# model2= Model(inputs=members[1].input, outputs=members[1].get_layer('quant_dense_26').output)
# model3= Model(inputs=members[2].input, outputs=members[2].get_layer('quant_dense_28').output)
# model4= Model(inputs=members[3].input, outputs=members[3].get_layer('quant_dense_30').output)
# model5= Model(inputs=members[4].input, outputs=members[4].get_layer('quant_dense_32').output)
# model6= Model(inputs=members[5].input, outputs=members[5].get_layer('quant_dense_36').output)
# model7= Model(inputs=members[6].input, outputs=members[6].get_layer('quant_dense_38').output)
# model8= Model(inputs=members[7].input, outputs=members[7].get_layer('quant_dense_40').output)


# model1= Model(inputs=members[0].input, outputs=members[0].get_layer('quant_dense_25').output)
# model2= Model(inputs=members[1].input, outputs=members[1].get_layer('quant_dense_27').output)
# model3= Model(inputs=members[2].input, outputs=members[2].get_layer('quant_dense_29').output)
# model4= Model(inputs=members[3].input, outputs=members[3].get_layer('quant_dense_31').output)
# model5= Model(inputs=members[4].input, outputs=members[4].get_layer('quant_dense_33').output)
# model6= Model(inputs=members[5].input, outputs=members[5].get_layer('quant_dense_37').output)
# model7= Model(inputs=members[6].input, outputs=members[6].get_layer('quant_dense_39').output)
# model8= Model(inputs=members[7].input, outputs=members[7].get_layer('quant_dense_41').output)




In [ ]:
# for i in range(len(model1.layers)):
#     model1.layers[i].trainable = False
#     model2.layers[i].trainable = False
#     model3.layers[i].trainable = False
#     model4.layers[i].trainable = False
#     model5.layers[i].trainable = False
#     model6.layers[i].trainable = False
#     model7.layers[i].trainable = False
#     model8.layers[i].trainable = False

In [ ]:
# x = layers.concatenate([model1.outputs[0], model2.outputs[0], model3.outputs[0], model4.outputs[0],
#                        model5.outputs[0], model6.outputs[0], model7.outputs[0], model8.outputs[0]])

# x = layers.BatchNormalization()(x) 
# x = layers.Dropout(0.2)(x)  
# outputs = layers.Dense(num_classes, activation='softmax')(x)  

# model = Model([model1.inputs[0], model2.inputs[0], model3.inputs[0], model4.inputs[0],
#               model5.inputs[0], model6.inputs[0], model7.inputs[0], model8.inputs[0]], outputs)

# model.compile(optimizer='adam',
#           loss='categorical_crossentropy',
#               metrics=['accuracy'])

In [ ]:
# model.summary()

In [ ]:
# model = tf.keras.models.load_model('vanilla_cifar10_bnn_supernet.h5')

In [ ]:
# lq.models.ModelProfile(model).generate_table()[-1][-3]

In [ ]:
# model.fit([x_train, x_train, x_train, x_train, x_train, x_train, x_train, x_train],
#           y_train, batch_size=256, epochs=100, validation_data=([x_test, x_test, x_test, x_test,
#                                                                 x_test, x_test, x_test, x_test], y_test))

In [ ]:
# model.save('vanilla_cifar10_bnn_supernet.h5')

In [ ]:
# len(members)

In [ ]:
# results = {}

In [ ]:
# a = model.predict([x_test, x_test, x_test, x_test, x_test, x_test, x_test, x_test])
# results['base'] = full_evaluate2(pred=a, y=y_test)
# results['base']['size (kb)'] = lq.models.ModelProfile(model).generate_table()[-1][-3]

In [ ]:
with open('results_bnn_supernet.json', 'r', encoding ='utf8') as json_file:
    results = json.load(json_file)

In [ ]:
results

# BNN + prunning

In [ ]:
def evaluate_ensamble_pruned(ensamble, x_test, y_test):
    y_pred = [model.predict_on_batch(x_test) for model in ensamble]
    a = np.mean(y_pred, axis=0)
    return full_evaluate2(pred=a, y=y_test)

In [ ]:
for i in range(8):
    try:
        for j, m in enumerate(members):
            members[j] = prune_model(m, 0.1, opt='adam', method='l1')
    except:
        break
    for j, m in enumerate(members):
        idx = resample(range(n), replace=False, n_samples=n*2//3)
        members[j].fit(x_train[idx], y_train[idx], batch_size=256, epochs=15,
              validation_data=(x_test, y_test))
    
    model1= Model(inputs=members[0].input, outputs=members[0].get_layer('quant_dense_24').output)
    model2= Model(inputs=members[1].input, outputs=members[1].get_layer('quant_dense_26').output)
    model3= Model(inputs=members[2].input, outputs=members[2].get_layer('quant_dense_28').output)
    model4= Model(inputs=members[3].input, outputs=members[3].get_layer('quant_dense_30').output)
    model5= Model(inputs=members[4].input, outputs=members[4].get_layer('quant_dense_32').output)
    model6= Model(inputs=members[5].input, outputs=members[5].get_layer('quant_dense_36').output)
    model7= Model(inputs=members[6].input, outputs=members[6].get_layer('quant_dense_38').output)
    model8= Model(inputs=members[7].input, outputs=members[7].get_layer('quant_dense_40').output)
    
    for j in range(len(model1.layers)):
        model1.layers[j].trainable = False
        model2.layers[j].trainable = False
        model3.layers[j].trainable = False
        model4.layers[j].trainable = False
        model5.layers[j].trainable = False
        model6.layers[j].trainable = False
        model7.layers[j].trainable = False
        model8.layers[j].trainable = False
    
    x = layers.concatenate([model1.outputs[0], model2.outputs[0], model3.outputs[0], model4.outputs[0],
                       model5.outputs[0], model6.outputs[0], model7.outputs[0], model8.outputs[0]])
    x = layers.BatchNormalization()(x) 
    x = layers.Dropout(0.2)(x)  
    outputs = layers.Dense(num_classes, activation='softmax')(x)  

    model = Model([model1.inputs[0], model2.inputs[0], model3.inputs[0], model4.inputs[0],
                  model5.inputs[0], model6.inputs[0], model7.inputs[0], model8.inputs[0]], outputs)

    model.compile(optimizer='adam',
              loss='categorical_crossentropy',
                  metrics=['accuracy'])
    model.fit([x_train, x_train, x_train, x_train, x_train, x_train, x_train, x_train],
          y_train, batch_size=256, epochs=10, validation_data=([x_test, x_test, x_test, x_test,
                                                                x_test, x_test, x_test, x_test], y_test))
    
    a = model.predict([x_test, x_test, x_test, x_test, x_test, x_test, x_test, x_test])
    results['bnn_node_pruning_{}'.format((0.9)**(i+1))] = full_evaluate2(pred=a, y=y_test)
    results['bnn_node_pruning_{}'.format((0.9)**(i+1))]['size (kb)'] = lq.models.ModelProfile(model).generate_table()[-1][-3]
    with open('results_bnn_supernet.json', 'w', encoding ='utf8') as json_file:
        json.dump(results, json_file, ensure_ascii = True)